In [ ]:
import csv
import os
import random
import pickle
import gc
import xgboost as xgb
import pandas as pd
import numpy as np
from scipy import stats
import sklearn
import warnings
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

# Stacking

## Body

In [ ]:
# OUTER STACK (PC version) RandomForest ensemble
# Configuration
algorithms = ['rf'] 
folds = range(5)  # Changed from strings to integers 0-4
modality_name = 'body'
seed = 10

base_path = '/UK_BB/brainbody'  
stacking_path = os.path.join(base_path, 'stacking', modality_name)
cognition_path = os.path.join(base_path, 'cognition')
merge_type = 'outer'

# Main processing loop
for fold in folds:
    print(f"\n{'='*50}")
    print(f"Processing fold {fold}")
    print(f"{'='*50}")
    
    # Create fold-specific paths
    folds_path = os.path.join(stacking_path, 'folds', f'fold_{fold}')
    suppl_path = os.path.join(folds_path, 'suppl')
    scaling_path = os.path.join(folds_path, 'scaling')
    models_path = os.path.join(folds_path, 'models')
    g_pred_path = os.path.join(folds_path, 'g_pred')

    # Create directories
    for path in [folds_path, suppl_path, scaling_path, models_path, g_pred_path]:
        os.makedirs(path, exist_ok=True)

    def load_data(fold, data_type):
        """Load pre-matched features and target data."""
        if data_type == 'train':
            data_path = os.path.join(stacking_path, 'features_train_level1_stacked_outer', 
                                   f'features_train_level1_outer_g_matched_fold_{fold}.csv')
        elif data_type == 'test':
            data_path = os.path.join(stacking_path, 'features_test_level1_stacked_outer',
                                   f'features_test_level1_outer_g_matched_fold_{fold}.csv')
        
        if not os.path.exists(data_path):
            raise FileNotFoundError(f"Data file not found at: {data_path}")
        
        df = pd.read_csv(data_path)
        print(f"\nLoading {data_type} data for fold {fold} from: {data_path}")
        print(f"Columns: {df.columns.tolist()}")
        print(f"Shape: {df.shape}")
        
        target_values = df['g'].values.reshape(-1, 1)
        target_ids = df['eid']
        features = df.drop(columns=['eid', 'g'])
        features.columns = features.columns.str.replace(f'_{data_type}_', '')
        
        return features, target_values, target_ids

    # Load train and test data
    features_train, target_real_train, target_real_train_id = load_data(fold, 'train')
    features_test, target_real_test, target_real_test_id = load_data(fold, 'test')

    # Scaling
    scaler_features = StandardScaler()
    scaler_target = StandardScaler()

    target_real_train = scaler_target.fit_transform(target_real_train)
    target_real_test = scaler_target.transform(target_real_test)

    features_train = scaler_features.fit_transform(features_train)
    features_test = scaler_features.transform(features_test)

    for algorithm in range(len(algorithms)):
        algorithm_performance = {}

        if algorithms[algorithm] == 'xgb':
            param = {
                'booster': ['gbtree'],
                'eta': [0.01, 0.05, 0.1],
                'max_depth': [1, 2, 3],
                'subsample': [0.8, 1],
                'reg_lambda': [0, 0.1, 0.5],
                'reg_alpha': [0, 0.01, 0.1],
                'min_child_weight': [1, 3],
                'colsample_bytree': [0.6, 0.8],
                'gamma': [0, 0.1]
            }
            regressor = xgb.XGBRegressor(
                random_state=seed,
                objective='reg:squarederror',
            )
         
        elif algorithms[algorithm] == 'rf':
            print('Started RF')
            param = {
                'n_estimators': [5000],
                'max_depth': [1, 2, 3, 4, 5, 6],
                'max_features': ['sqrt', 'log2']
            }
            regressor = RandomForestRegressor(random_state=seed)

        model = GridSearchCV(regressor, param, cv=5, verbose=4, n_jobs=-1)  # n_jobs=-1 uses all cores
        model.fit(features_train, target_real_train.ravel())

        # Save model
        with open(os.path.join(models_path, f'{modality_name}_stacked_{algorithm}_{merge_type}_model_fold_{fold}.pkl'), 'wb') as f:
            pickle.dump(model, f)

        # Predictions
        target_pred_train = model.predict(features_train)
        target_pred_test = model.predict(features_test)

        # Save predictions
        pd.DataFrame({'eid': target_real_train_id, 'g_pred_stack_train': target_pred_train}).to_csv(
            os.path.join(g_pred_path, f'{modality_name}_target_pred_2nd_level_{algorithm}_{merge_type}_train_fold_{fold}.csv'),
            index=False
        )

        pd.DataFrame({'eid': target_real_test_id, 'g_pred_stack_test': target_pred_test}).to_csv(
            os.path.join(g_pred_path, f'{modality_name}_target_pred_2nd_level_{algorithm}_{merge_type}_test_fold_{fold}.csv'),
            index=False
        )

        # Store results
        algorithm_performance = {
            'Algorithm': algorithms[algorithm],
            'Fold': fold,
            'Best_params': str(model.best_params_),
            'Test_MSE': mean_squared_error(target_real_test, target_pred_test),
            'Test_MAE': mean_absolute_error(target_real_test, target_pred_test),
            'Test_R2': r2_score(target_real_test, target_pred_test),
            'Test_Pearson_r': pearsonr(target_real_test.squeeze(), target_pred_test.squeeze())[0],
            'Train_MSE': mean_squared_error(target_real_train, target_pred_train),
            'Train_MAE': mean_absolute_error(target_real_train, target_pred_train),
            'Train_R2': r2_score(target_real_train, target_pred_train),
            'Train_Pearson_r': pearsonr(target_real_train.squeeze(), target_pred_train.squeeze())[0]
        }

        # Write results
        results_file = os.path.join(models_path, f'{modality_name}_{algorithm}_{merge_type}_stacked_result_fold_{fold}.csv')
        with open(results_file, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=algorithm_performance.keys())
            writer.writeheader()
            writer.writerow(algorithm_performance)

print("\nAll folds processed successfully!")

## Body + brain

In [ ]:
# OUTER STACK (PC version) RandomForest ensemble
# Configuration
algorithms = ['rf']
folds = range(5)
modality_name = 'brain-body'
seed = 10

base_path = '/UK_BB/brainbody'  
stacking_path = os.path.join(base_path, 'stacking', modality_name)
cognition_path = os.path.join(base_path, 'cognition')
merge_type = 'outer'

# Main processing loop
for fold in folds:
    print(f"\n{'='*50}")
    print(f"Processing fold {fold}")
    print(f"{'='*50}")
    
    # Create fold-specific paths
    folds_path = os.path.join(stacking_path, 'folds', f'fold_{fold}')
    suppl_path = os.path.join(folds_path, 'suppl')
    scaling_path = os.path.join(folds_path, 'scaling')
    models_path = os.path.join(folds_path, 'models')
    g_pred_path = os.path.join(folds_path, 'g_pred')

    # Create directories
    for path in [folds_path, suppl_path, scaling_path, models_path, g_pred_path]:
        os.makedirs(path, exist_ok=True)

    def load_data(fold, data_type):
        """Load pre-matched features and target data."""
        if data_type == 'train':
            data_path = os.path.join(stacking_path, 'features_train_level1_stacked_outer', 
                                   f'features_train_level1_outer_g_matched_fold_{fold}.csv')
        elif data_type == 'test':
            data_path = os.path.join(stacking_path, 'features_test_level1_stacked_outer',
                                   f'features_test_level1_outer_g_matched_fold_{fold}.csv')
        
        if not os.path.exists(data_path):
            raise FileNotFoundError(f"Data file not found at: {data_path}")
        
        df = pd.read_csv(data_path)
        print(f"\nLoading {data_type} data for fold {fold} from: {data_path}")
        print(f"Columns: {df.columns.tolist()}")
        print(f"Shape: {df.shape}")
        
        target_values = df['g'].values.reshape(-1, 1)
        target_ids = df['eid']
        features = df.drop(columns=['eid', 'g'])
        features.columns = features.columns.str.replace(f'_{data_type}_', '')
        
        return features, target_values, target_ids

    # Load train and test data
    features_train, target_real_train, target_real_train_id = load_data(fold, 'train')
    features_test, target_real_test, target_real_test_id = load_data(fold, 'test')

    # Scaling
    scaler_features = StandardScaler()
    scaler_target = StandardScaler()

    target_real_train = scaler_target.fit_transform(target_real_train)
    target_real_test = scaler_target.transform(target_real_test)

    features_train = scaler_features.fit_transform(features_train)
    features_test = scaler_features.transform(features_test)

    for algorithm in range(len(algorithms)):
        algorithm_performance = {}

        if algorithms[algorithm] == 'xgb':
            param = {
                'booster': ['gbtree'],
                'eta': [0.01, 0.05, 0.1],
                'max_depth': [1, 2, 3],
                'subsample': [0.8, 1],
                'reg_lambda': [0, 0.1, 0.5],
                'reg_alpha': [0, 0.01, 0.1],
                'min_child_weight': [1, 3],
                'colsample_bytree': [0.6, 0.8],
                'gamma': [0, 0.1]
            }
            regressor = xgb.XGBRegressor(
                random_state=seed,
                objective='reg:squarederror',
            )
         
        elif algorithms[algorithm] == 'rf':
            print('Started RF')
            param = {
                'n_estimators': [5000],
                'max_depth': [1, 2, 3, 4, 5, 6],
                'max_features': ['sqrt', 'log2']
            }
            regressor = RandomForestRegressor(random_state=seed)

        model = GridSearchCV(regressor, param, cv=5, verbose=4, n_jobs=-1)  # n_jobs=-1 uses all cores
        model.fit(features_train, target_real_train.ravel())

        # Save model
        with open(os.path.join(models_path, f'{modality_name}_stacked_{algorithm}_{merge_type}_model_fold_{fold}.pkl'), 'wb') as f:
            pickle.dump(model, f)

        # Predictions
        target_pred_train = model.predict(features_train)
        target_pred_test = model.predict(features_test)

        # Save predictions
        pd.DataFrame({'eid': target_real_train_id, 'g_pred_stack_train': target_pred_train}).to_csv(
            os.path.join(g_pred_path, f'{modality_name}_target_pred_2nd_level_{algorithm}_{merge_type}_train_fold_{fold}.csv'),
            index=False
        )

        pd.DataFrame({'eid': target_real_test_id, 'g_pred_stack_test': target_pred_test}).to_csv(
            os.path.join(g_pred_path, f'{modality_name}_target_pred_2nd_level_{algorithm}_{merge_type}_test_fold_{fold}.csv'),
            index=False
        )

        # Store results
        algorithm_performance = {
            'Algorithm': algorithms[algorithm],
            'Fold': fold,
            'Best_params': str(model.best_params_),
            'Test_MSE': mean_squared_error(target_real_test, target_pred_test),
            'Test_MAE': mean_absolute_error(target_real_test, target_pred_test),
            'Test_R2': r2_score(target_real_test, target_pred_test),
            'Test_Pearson_r': pearsonr(target_real_test.squeeze(), target_pred_test.squeeze())[0],
            'Train_MSE': mean_squared_error(target_real_train, target_pred_train),
            'Train_MAE': mean_absolute_error(target_real_train, target_pred_train),
            'Train_R2': r2_score(target_real_train, target_pred_train),
            'Train_Pearson_r': pearsonr(target_real_train.squeeze(), target_pred_train.squeeze())[0]
        }

        # Write results
        results_file = os.path.join(models_path, f'{modality_name}_{algorithm}_{merge_type}_stacked_result_fold_{fold}.csv')
        with open(results_file, 'w', newline='') as f:
            writer = csv.DictWriter(f, fieldnames=algorithm_performance.keys())
            writer.writeheader()
            writer.writerow(algorithm_performance)

print("\nAll folds processed successfully!")


Processing fold 0

Loading train data for fold 0 from: /media/hcs-sci-psy-narun/IBu/UK_BB/brainbody/stacking/brain-body/features_train_level1_stacked_outer/features_train_level1_outer_g_matched_fold_0.csv
Columns: ['eid', 'g_pred_train_immune', 'g_pred_train_renalhepatic', 'g_pred_train_metabolic', 'g_pred_train_cardiopulmonary', 'g_pred_train_musculoskeletal', 'g_pred_train_bone_densitometry', 'g_pred_train_pwa', 'g_pred_train_heart_mri', 'g_pred_train_carotid_ultrasound', 'g_pred_train_arterial_stiffness', 'g_pred_train_ecg_rest', 'g_pred_train_body_composition_by_impedance', 'g_pred_train_body_composition_dxa', 'g_pred_train_bone_dxa', 'g_pred_train_kidneys_mri', 'g_pred_train_liver_mri', 'g_pred_train_abdominal_composition_mri_18_vars', 'g_pred_train_abdominal_organ_composition_mri_13_vars', 'g_pred_train_hearing', 'g_pred_train_struct_fast', 'g_pred_train_struct_sub_first', 'g_pred_train_struct_fs_aseg_mean_intensity', 'g_pred_train_struct_fs_aseg_volume', 'g_pred_train_struct_ba